In [7]:
import os
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import Dataset
import EIDA


tokenizer = GPT2Tokenizer.from_pretrained("gpt2") # 사전 크기: 50257
model = GPT2LMHeadModel.from_pretrained("gpt2", device_map='auto') # layer 12개, d_model 768차원
tokenizer.pad_token = tokenizer.eos_token


# E2E NLG Challenge 데이터셋의 예시:
# name : Blue Spice | Type : coffee shop | area : city centre||A coffee shop in the city centre area called Blue Spice .
# '||' 이전 부분이 모델에 입력으로 주어지는 context(meaning representation 속성)
# '||' 이후 부분이 모델이 생성할 자연어 completion(human_reference 속성)
# E2E 폴더에 train.txt, valid.txt, test.txt 파일이 들어있어야 실행됨.
data = []
with open(os.path.join("E2E", "train.txt"), 'r', encoding='utf8') as f:
	for line in f:
		meaning_representation, human_reference = line.strip().split('||')
		data.append({'meaning_representation': meaning_representation, 'human_reference': human_reference})
train_dataset = Dataset.from_list(data)

data = []
with open(os.path.join("E2E", "valid.txt"), 'r', encoding='utf8') as f:
	for line in f:
		meaning_representation, human_reference = line.strip().split('||')
		data.append({'meaning_representation': meaning_representation, 'human_reference': human_reference})
valid_dataset = Dataset.from_list(data)

data = []
with open(os.path.join("E2E", "test.txt"), 'r', encoding='utf8') as f:
	for line in f:
		meaning_representation, human_reference = line.strip().split('||')
		data.append({'meaning_representation': meaning_representation, 'human_reference': human_reference})
test_dataset = Dataset.from_list(data)

In [2]:
max_seq_len = 144 # 모든 데이터를 잘라내지 않고 다룰 수 있는 충분한 길이

tokenized_data = []
for example in train_dataset:
    tokenized_meaning_representation = tokenizer.encode(example['meaning_representation'] + tokenizer.bos_token)
    tokenized_human_reference = tokenizer.encode(' ' + example['human_reference'] + tokenizer.eos_token)
    # GPT2의 input을 (meaning representation + BOS + ' ' + human reference + EOS)로 구성하는 것은 microsoft/LoRA 깃헙에서 입력을 구성한 방식을 그대로 따랐음.
    # GPT2에서는 BOS와 EOS 둘 다 50256번 토큰 <|endoftext|>으로 되어있고, PAD 토큰은 별도로 지정되어 있지 않음.
    # 현재 PAD 토큰을 50256으로 설정해서 사용하고 있음. padding한 위치들에는 labels값 -100을 설정해서 학습과정에 관여하지 않게 만들 것이므로 PAD 토큰을 50256으로 설정한 것이 학습에 영향을 미치지 않음.

    input_ids = tokenized_meaning_representation + tokenized_human_reference + [tokenizer.pad_token_id for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))] # context + completion + padding 구성으로 길이 max_seq_len만큼의 시퀀스

    labels = [-100 for _ in tokenized_meaning_representation] + tokenized_human_reference + [-100 for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]
    # labels에 -100이 들어있는 자리는 loss function이 무시하도록 설정되어 있다.(이것은 허깅페이스 transformers 패키지의 설정이 아니고 torch.nn.CrossEntropyLoss의 디폴트 설정임)
    # 모델이 context(meaning representation)를 입력받아 completion(human reference)을 생성하는 학습을 수행할 것이므로, labels에서 completion 부분의 토큰들만 남겨놓고 context 부분과 padding 부분은 -100으로 설정해주어야 의도한 학습을 수행하게 된다.
    
    tokenized_data.append({'input_ids': input_ids, 'labels': labels})
preprocessed_train_dataset = Dataset.from_list(tokenized_data)


tokenized_data = []
for example in valid_dataset:
    tokenized_meaning_representation = tokenizer.encode(example['meaning_representation'] + tokenizer.bos_token)
    tokenized_human_reference = tokenizer.encode(' ' + example['human_reference'] + tokenizer.eos_token)

    input_ids = tokenized_meaning_representation + tokenized_human_reference + [tokenizer.pad_token_id for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    labels = [-100 for _ in tokenized_meaning_representation] + tokenized_human_reference + [-100 for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    tokenized_data.append({'input_ids': input_ids, 'labels': labels})
preprocessed_valid_dataset = Dataset.from_list(tokenized_data)


tokenized_data = []
for example in test_dataset:
    tokenized_meaning_representation = tokenizer.encode(example['meaning_representation'] + tokenizer.bos_token)
    tokenized_human_reference = tokenizer.encode(' ' + example['human_reference'] + tokenizer.eos_token)

    input_ids = tokenized_meaning_representation + tokenized_human_reference + [tokenizer.pad_token_id for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    labels = [-100 for _ in tokenized_meaning_representation] + tokenized_human_reference + [-100 for _ in range(max_seq_len - len(tokenized_meaning_representation) - len(tokenized_human_reference))]

    tokenized_data.append({'input_ids': input_ids, 'labels': labels})
preprocessed_test_dataset = Dataset.from_list(tokenized_data)

In [ ]:
# 원본 모델의 파라미터는 모두 고정하고, 추가할 어댑터 안에서만 학습가능한 파라미터를 두려고 함.
# 각 블록 안의 c_attn(W_Q, W_K, W_V를 합쳐둔 가중치), c_proj(W_O에 해당하는 가중치), c_fc(FFN 안의 W_fc1에 해당하는 가중치), c_proj(FFN 안의 W_fc2에 해당하는 가중치)는 뒤에서 EIDA.Linear_with_adapter 타입으로 교체할 때 requires_grad=False 설정이 이루어짐. 여기에 해당하지 않는 학습가능한 파라미터인 embedding과 layer normalization은 지금 고정함.
model.transformer.wte.requires_grad = False
model.transformer.wpe.requires_grad = False
for block in model.transformer.h: # 모델의 attention block 12개
    block.ln_1.requires_grad = False
    block.ln_2.requires_grad = False
model.transformer.ln_f.requires_grad = False


for _ in range(16): # 16에폭의 학습을 수행하기 위한 반복문. 이 반복문은 다음과 같이 구성됨
    # 1. 72개의 파라미터 W 각각에 대해, W의 input token X와 Δoutput(학습과정에서 가중치 W의 gradient인 ΔW에 의한 output token Y의 변화량 ΔY=ΔW×Y를 의미)의 표본을 추출함. EIDA.forward_gpt2 함수는 표본 토큰들을 sample_inputs, sample_delta_outputs 배열로 반환함.
    # 2. 추출된 표본을 가지고 PCA를 수행하여, E2E NLG Challenge train set의 데이터를 GPT2에 통과시킬 때 각 latent space에서 나타나는 토큰의 분포를 저차원(32나 64) 부분공간으로 근사하여 표현함. EIDA.PCA 함수는 전체 latent space에서 추정된 저차원 부분공간으로 가는 projection map을 반환하도록 만들어져 있는데, 이것을 plane_inputs, plane_delta_outputs 배열에 저장함.
    # 3. 각 W에 대해, (W의 정의역에서 부분공간으로 가는 projection map), (X의 분포가 집중된 부분공간에서 ΔY의 분포가 집중된 부분공간으로 가는 학습 가능한 파라미터), (ΔY의 분포가 집중된 부분공간에서 공역으로 가는 projection map)^T 을 합성한 형태로 어댑터를 구성함. EIDA.Conv1D_with_adapter 타입이 이 형태를 지니도록 만들어짐.
    # 4. Hugging Face 패키지의 trainer를 이용해서 1에폭 학습을 수행함. 1에폭 단위로 linear schedule(처음에 warmup 구간, 이후에 decay 구간)을 따르게 됨.

    preprocessed_train_dataset = preprocessed_train_dataset.shuffle() # train set 순서 섞기
    input_ids = torch.tensor(preprocessed_train_dataset['input_ids'])
    labels = torch.tensor(preprocessed_train_dataset['labels'])

    sample_inputs, sample_delta_outputs = EIDA.forward_gpt2(model, input_ids, labels, begin=0, end=128, batch_size=2, max_length=max_seq_len, p=0.0075)

    plane_inputs=[]
    # 표본추출을 수행한 공간들(input token들이 머무르는 latent space 48개)을 가리키는 index:
    # 4*l+0: layer[l]에서 W_Q, W_K, W_V들의 공통 input의 위치 (768차원) (l = 0, 1, ..., 11)
    # 4*l+1: layer[l]에서 W_O의 input의 위치 (768차원)
    # 4*l+2: layer[l]에서 W_fc1의 input의 위치 (768차원)
    # 4*l+3: layer[l]에서 W_fc2의 input의 위치 (3072차원)
    list_input_64 = [0, 2, 3, 4, 6, 7, 10, 11, 14, 15, 17, 18, 19, 20, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 38, 39, 40, 42, 43, 44, 46, 47] # 차원을 64로 둘 input 공간들의 index
    for i in range(4*12):
        plane_inputs.append(EIDA.PCA(sample_inputs[i], begin=0, end=len(sample_inputs[i]), plane_dim=64 if i in list_input_64 else 32, device=model.device)) # 주어지는 토큰들로 주성분분석을 수행해서, 768(또는 3072)차원 공간에서 token representation의 분포를 가장 잘 포착하는 32(또는 64)차원원 평면을 추정하는 함수
    del sample_inputs
    # plane_inputs는 [32(또는 64), 768(또는 3072)]의 shape을 가진 torch.tensor 64개의 list
    # 각 텐서는 latent space에서 PCA를 통해 추산된 32(또는 64)차원 부분공간으로의 projection map을 의미하고, 벡터 32(또는 64)개가 서로 orthonormal하게 있음. 이 맵이 어댑터의 A를 구성함.

    plane_delta_outputs=[]
    # 표본추출을 수행한 공간들(output token들이 머무르는 latent space 72개)을 가리기는 index:
    # 6*l+0: layer[l]에서 W_Q의 output의 위치 (768차원)
    # 6*l+1: layer[l]에서 W_K의 output의 위치 (768차원)
    # 6*l+2: layer[l]에서 W_V의 output의 위치 (768차원)
    # 6*l+3: layer[l]에서 W_O의 output의 위치 (768차원)
    # 6*l+4: layer[l]에서 W_fc1의 output의 위치 (3072차원)
    # 6*l+5: layer[l]에서 W_fc2의 output의 위치 (768차원)
    list_delta_output_64 = [0, 1, 3, 5, 9, 10, 11, 14, 15, 17, 20, 21, 23, 26, 27, 32, 38, 44, 56, 62, 64, 65, 69, 70, 71] # 차원을 64로 둘 output 공간들의 index
    for i in range(6*12):
        plane_delta_outputs.append(EIDA.PCA(sample_delta_outputs[i], begin=0, end=len(sample_delta_outputs[i]), plane_dim=64 if i in list_delta_output_64 else 32, device=model.device))
    del sample_delta_outputs
    # plane_delta_outputs는 [32(또는 64), 768(또는 3072)]의 shape을 가진 torch.tensor 72개의 list
    # 각 텐서는 latent space에서 PCA를 통해 추산된 32(또는 64)차원 부분공간으로의 projection map을 의미하고, 벡터 32(또는 64)개가 서로 orthonormal하게 있음. 이 맵의 transpose가 어댑터의 C를 구성함.


    # 모델 안의 72개의 파라미터를 각각 EIDA.Linear_with_adapter 타입으로 교체하는 과정
    # EIDA.Linear_with_adapter는 원본 파라미터의 weight을 (원본 파라미터의 weight) + (A @ B @ C) 로 바꿈(bias가 있다면 그대로 가져옴). 이 중에서 B만 학습가능한 파라미터로 둔다.
    # A는 파라미터의 input token이 머무는 latent space에서 32(또는 64)차원 부분공간으로의 projection map.
    # C는 파라미터의 output token이 머무는 latent space에서 32(또는 64)차원 부분공간으로의 projection map을 transpose해서 얻어진, 32(또는 64)차원에서 768(또는 3072)차원으로 가는 map.(각 열의 orthonormality 덕분에 transpose가 projection의 역과정이 됨)
    # B는 그 둘을 잇는, 32(또는 64)차원에서 32(또는 64)차원으로 가는 linear map.
    for l in range(len(model.transformer.h)):
        block = model.transformer.h[l]

        block.attn.c_attn = EIDA.Conv1D_with_adapter_for_c_attn(original_param=block.attn.c_attn, A=plane_inputs[4*l+0], C_Q=plane_delta_outputs[6*l+0], C_K=plane_delta_outputs[6*l+1], C_V=plane_delta_outputs[6*l+2])
        block.attn.c_proj = EIDA.Conv1D_with_adapter(original_param=block.attn.c_proj, A=plane_inputs[4*l+1], C=plane_delta_outputs[6*l+3])
        block.mlp.c_fc = EIDA.Conv1D_with_adapter(original_param=block.mlp.c_fc, A=plane_inputs[4*l+2], C=plane_delta_outputs[6*l+4])
        block.mlp.c_proj = EIDA.Conv1D_with_adapter(original_param=block.mlp.c_proj, A=plane_inputs[4*l+3], C=plane_delta_outputs[6*l+5])

    training_args = TrainingArguments(
        output_dir=os.path.join("results", "gpt-2", "e2e-nlg"),
        save_strategy="no",
        num_train_epochs=1,
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        learning_rate=5e-5,
        warmup_ratio=0.15,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=525,
        eval_strategy="steps",
        eval_steps=525,
        fp16=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=preprocessed_train_dataset,
        eval_dataset=preprocessed_valid_dataset
    )

    trainer.train()

    # 학습된 어댑터를 모델에 merge하고 모델을 저장하기
    for l in range(len(model.transformer.h)):
        block = model.transformer.h[l]

        block.attn.c_attn = block.attn.c_attn.merge()
        block.attn.c_proj = block.attn.c_proj.merge()
        block.mlp.c_fc = block.mlp.c_fc.merge()
        block.mlp.c_proj = block.mlp.c_proj.merge()

    model.save_pretrained(os.path.join("results", "gpt-2", "e2e-nlg", f"{_}"))

In [ ]:
list_test = []
# test set에서는 하나의 context('meaning_representation')에 대해 여러 개의 reference('human_reference')를 제공함. 예시로 test set의 첫 두 행은 다음과 같음:
# name : Blue Spice | Type : coffee shop | area : city centre||A coffee shop in the city centre area called Blue Spice . 
# name : Blue Spice | Type : coffee shop | area : city centre||Blue Spice is a coffee shop in city centre . 
# 그래서 이 대응을 {context: (context 문장), references: [(completion 문장1), (completion 문장2), ...]} 형태로 변환해서 list_test에 넣을 거임. test set에서 같은 context를 가진 행은 한 곳에 잘 모여있다.
for i in range(len(test_dataset)):
    if len(list_test) == 0 or list_test[-1]['context'] != test_dataset[i]['meaning_representation']:
        list_test.append({'context': test_dataset[i]['meaning_representation'], 'references': [test_dataset[i]['human_reference']]})
    else:
        list_test[-1]['references'].append(test_dataset[i]['human_reference'])

print(len(list_test)) # 총 4693행을 가진 test set에 대해, 630이 출력됨.


# 모델에게 test set의 630개 context에 대해 출력을 생성하게 하는 과정. 출력된 문장은 list_test의 각 dict에 'prediction' 속성으로 추가됨.
model.eval()
for i in range(len(list_test)):
    encoded = tokenizer(list_test[i]['context'] + tokenizer.bos_token, return_tensors='pt').to(model.device)
    output = model.generate(
        input_ids=encoded['input_ids'],
        attention_mask=encoded['attention_mask'],
        max_length=max_seq_len,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        num_beams=10, # beam search (weight 10)
        early_stopping=True
    )
    output = tokenizer.decode(output[0])
    list_test[i]['prediction'] = output[len(list_test[i]['context']):]


# 모델이 context를 입력받아서 context + completion을 출력해놨는데 context 부분을 떼어내고 completion 부분만 list_prediction에다 모으는 과정.
def save_benchmark_data(list_references, list_prediction, reference_file, prediction_file):
    with open(reference_file, "w", encoding="utf-8") as ref_file:
        for refs in list_references:
            for ref in refs:
                ref_file.write(ref + "\n")
            ref_file.write("\n")

    with open(prediction_file, "w", encoding="utf-8") as pred_file:
        for pred in list_prediction:
            pred_file.write(pred + "\n")

            
list_references = []
list_prediction = []

for i in range(len(list_test)):
    list_references.append(list_test[i]['references'])
    list_prediction.append(list_test[i]['prediction'][len(tokenizer.bos_token)+1:-len(tokenizer.eos_token)])

save_benchmark_data(list_references, list_prediction, "references.txt", "predictions_0.txt")
# 이제 출력된 두 txt 파일을 https://github.com/tuetschek/e2e-metrics 에서 제공하는 벤치마크에 넣으면 된다.